In [1]:
import numpy as np
from Ausgleichsbecken_class_file import Ausgleichsbecken_class
import matplotlib.pyplot as plt

#importing pressure conversion function
import sys
import os
current = os.path.dirname(os.path.realpath('Main_Programm.ipynb'))
parent = os.path.dirname(current)
sys.path.append(parent)
from functions.pressure_conversion import pressure_conversion

In [2]:
# define constants
initial_level               = 10.        # m
initial_influx              = 5.       # m³/s
# initial_outflux             = 1.        # m³/s
# initial_pipeline_pressure   = 10.
# initial_pressure_unit       = 'mWS'
conversion_pressure_unit    = 'mWS'

area_base                   = 1.        # m²
area_outflux                = 0.5       # m²
critical_level_low          = 0.        # m
critical_level_high         = 10.       # m
simulation_timestep         = 0.001     # s

# for while loop
total_min_level             = 0.01      # m
total_max_time              = 1000     # s

In [3]:
%matplotlib qt

V = Ausgleichsbecken_class(area_base, area_outflux, critical_level_low, critical_level_high,simulation_timestep)
# V.set_initial_level(initial_level) 
# V.set_influx(initial_influx)
# V.set_outflux(initial_outflux)
# V.set_initial_pressure(pressure_conversion(initial_pipeline_pressure,input_unit = initial_pressure_unit, target_unit = 'Pa'),conversion_pressure_unit)
# V.pressure = converted_pressure
V.set_steady_state(initial_influx,initial_level,conversion_pressure_unit)

time_vec        = np.arange(0,total_max_time,simulation_timestep)
outflux_vec     = np.empty_like(time_vec)
outflux_vec[0]  = V.get_current_outflux()
level_vec       = np.empty_like(time_vec)
level_vec[0]    = V.get_current_level()

# pressure_vec    = np.full_like(time_vec,converted_pressure)*((np.sin(time_vec)+1)*np.exp(-time_vec/50))
pressure_vec    = np.full_like(time_vec,V.get_current_pressure())
 
i_max = -1

for i in range(np.size(time_vec)-1):
    V.set_pressure(pressure_vec[i])
    V.timestep_reservoir_evolution()
    outflux_vec[i+1] = V.get_current_outflux()
    level_vec[i+1]   = V.get_current_level()
    if V.level < total_min_level:
        i_max = i
        break



In [7]:

fig1, (ax1, ax2, ax3) = plt.subplots(3, 1)
fig1.set_figheight(10)
fig1.suptitle('Ausgleichsbecken')

ax1.plot(time_vec[:i_max],level_vec[:i_max], label='Water level')
ax1.set_ylabel(r'$h$ ['+V.level_unit+']')
ax1.set_xlabel(r'$t$ ['+V.time_unit+']')
ax1.legend()

ax2.plot(time_vec[:i_max],outflux_vec[:i_max], label='Outflux')
ax2.set_ylabel(r'$Q_{out}$ ['+V.flux_unit+']')
ax2.set_xlabel(r'$t$ ['+V.time_unit+']')
ax2.legend()

ax3.plot(time_vec[:i_max],pressure_conversion(pressure_vec[:i_max],'Pa',conversion_pressure_unit), label='Pipeline pressure at reservoir')
ax3.set_ylabel(r'$p_{pipeline}$ ['+conversion_pressure_unit+']')
ax3.set_xlabel(r'$t$ ['+V.time_unit+']')
ax3.legend()


fig1.tight_layout()            